# parsing хайтек

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm.notebook import trange, tqdm
from tqdm.gui import tqdm as tqdm_gui
from IPython.display import display

импорт библиотек

In [ ]:
import warnings
warnings.filterwarnings('ignore')

отключение ворнингов

In [ ]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

задание параметров таблиц

In [ ]:
reqx = requests.get('https://hi-tech.md/televizory-i-elektronika/page-1/')

In [ ]:
soup = bs(reqx.text, 'html.parser')

In [ ]:
data = pd.DataFrame({'soup_name':pd.Series(soup.findAll('div', class_='ypi-grid-list__item_body'),dtype = 'object')})
data

data.loc[0,'soup_name'].find('span', class_="ty-strike").text.replace(u'\xa0',u'').replace(u'руб',u'')

data.loc[5,'soup_name']

парсинг тестовой страници сайта 

def names_correct(x):
    name = str(x)
    return  name 

data['soup_name'] = data['soup_name'].apply(names_correct)

меняем данные парсинга на текстовый тип 

In [ ]:
def find_id(x):
    xxx = x.find('span', class_="ty-control-group__item").text  
    return xxx

In [ ]:
data['id'] = data['soup_name'].apply(find_id)

находим в тексте и созаем столбец с кодами товара

In [ ]:
data['date'] = pd.to_datetime('today').normalize()

создаем столбец с датой парсинга

In [ ]:
def find_availability(t):
    xxx = t.find('a', class_="ty-control-group product-list-field cm-external-click").text.replace('\ue924','').replace('\n','')
    return xxx

In [ ]:
data['availability'] = data['soup_name'].apply(find_availability)

находим в тексте и создаем столбец с наличием товара

In [ ]:
def find_title(t):
    xxx = t.find('a', class_="product-title").text
    return xxx

In [ ]:
data['title'] = data['soup_name'].apply(find_title)

находим в тексте и создаем столбец снаименованием товара

In [ ]:
def find_price(t):
    xxx = t.find('span', class_="ty-price-num").text.replace(u'\xa0',u'')
    return xxx

In [ ]:
data['price'] = data['soup_name'].apply(find_price)

Находим в тексте и создаем столбец с ценой товара

In [ ]:
def find_old_price(t):
    try:
        xxx = t.find('span', class_="ty-strike").text.replace(u'\xa0',u'').replace(u'руб',u'')
        return xxx
    except:
        return np.NaN

In [ ]:
data['old_price'] = data['soup_name'].apply(find_old_price)

Для акционного товара Находим в тексте и создаем столбец со старой ценой до начала акции

In [ ]:
def find_url(t):
    xxx = t.find('a', class_="abt-single-image")['href']
    return xxx

In [ ]:
data['soup_name'] = data['soup_name'].apply(find_url)

Находим в тексте информацию об адресе совара и переписываем эту информацию вместо парсинговой.

In [ ]:
def preprocessing(x):
    print(f'таблица имеет следующий вид:')
    display(x.head())
    print('*'*100)
    print(f'таблица имеет {x.shape[0]} сторок и {x.shape[1]} столбцов')
    print('*'*100)
    print(f'в таблице обнаружено дублекатов:{x.duplicated().sum()}')
    print('*'*100)
    print('в таблице обнаружены пропуски в следующих столбцах:')
    display(x.isna().sum())
    print('*'*100)
    print('Эти пропуски составлябт следующее количество в процентах')
    display((x.isna().mean()* 100).round(1))
    print('*'*100)
    print('столбцы имеют следующие типы:')
    display(x.dtypes)
    x.columns = [y.lower().replace(' ', '_') for y in x.columns.values] 
    print('*'*100)
    print('названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание')
    print('*'*100)
    display(x.info())

создаем функцию предварительного просмотра данных

In [ ]:
def parsing_2(url):
    data = pd.DataFrame(columns=['soup_name', 'id', 'date', 'availability', 'title', 'price', 'old_price'])
    for i in tqdm(url):        
        q = 1
        while q>0:        
            reqx = requests.get(i + str(q))
            soup = bs(reqx.text, 'html.parser')
            datax = pd.DataFrame({'soup_name':pd.Series(soup.findAll('div', class_='ypi-grid-list__item_body'),dtype = 'object')})
            data = pd.concat([data, datax], ignore_index=True)
            if len(list(soup.findAll('div', class_='ypi-grid-list__item_body'))) > 0:
                q = q + 1
            else:
                q = 0
    #data['soup_name'] = data['soup_name'].apply(names_correct)
    data['id'] = data['soup_name'].apply(find_id)
    data['date'] = pd.to_datetime('today').normalize()
    data['availability'] = data['soup_name'].apply(find_availability)
    data['title'] = data['soup_name'].apply(find_title)
    data['price'] = data['soup_name'].apply(find_price)
    data['old_price'] = data['soup_name'].apply(find_old_price)
    data['soup_name'] = data['soup_name'].apply(find_url)
    data['price'] = data['price'].astype(float)
    data.loc[data['old_price'] == '','old_price'] = np.nan
    data['old_price'] = data['old_price'].astype(float)
    return data

создаем функцию, которая по списку url проходит и парсит всю доступную информацию, преобразуя ее в датафрейм.

all_url = ['https://hi-tech.md/televizory-i-elektronika/page-',
       'https://hi-tech.md/bytovaya-tehnika/page-',
       'https://hi-tech.md/kompyuternaya-tehnika/page-']

all_url = ['https://hi-tech.md/bytovaya-tehnika/page-']

In [ ]:
all_url = ['https://hi-tech.md/televizory-i-elektronika/page-',
           'https://hi-tech.md/bytovaya-tehnika/page-',
           'https://hi-tech.md/kompyuternaya-tehnika/page-',
           'https://hi-tech.md/mebel-tekstil/kompyuternaya-mebel/ofisnye-kresla-i-stulya/komp.-kresla/page-',
           'https://hi-tech.md/instrumenty-i-oborudovanie/elektroinstrument/page-',
           'https://hi-tech.md/tovary-dlya-doma/page-'
          ]

список url по которым будет парсится сайт

In [ ]:
df = parsing_2(all_url)
df

полученная информация

In [ ]:
preprocessing(df)

In [ ]:
 df = df.drop_duplicates()

In [ ]:
today = pd.to_datetime('today').normalize()
today = today.to_pydatetime()
df.to_csv('csv/hi_tech_test1_'+str(today.year)+'_'+str(today.month)+'_'+str(today.day)+'.csv', index=False)  

Переписывем датафрейм в файл

# аналитика хайтек

In [ ]:
brend = pd.read_excel('csv/brend.xlsx')

In [ ]:
def lower(x):
    return x.lower()

In [ ]:
brend['names'] = brend['name'].apply(lower)

In [ ]:
brend_up = brend.name.unique()
brend_low = brend.names.unique()

загружаем и создаем переменную с брендами

In [ ]:
df

In [ ]:
def category(x):
    xx = x.split('/')
    xx = xx[3].replace('-','_')
    return xx    

In [ ]:
def subcategory(x):
    xx = x.split('/')
    xx = xx[4].replace('-','_')
    return xx   

In [ ]:
def group(x):
    xx = x.split('/')
    if len(xx) == 7:
        return xx[-3].replace('-','_')
    elif len(xx) == 8:
        return xx[-3].replace('-','_')
    elif len(xx) == 9:
        return xx[-4].replace('-','_')
    elif len(xx) == 10:
        return xx[-5].replace('-','_')
    elif len(xx) == 11:
        return xx[-6].replace('-','_')  

создаем вспомогательные функции

In [ ]:
df['action_sale'] = 0

In [ ]:
df.loc[df['old_price'] > 0,'action_sale'] = 1

создаем столбец который показывает является ли товар акционным(1 - акционный, 0 - не акционный)

In [ ]:
df['sale'] = (100 - ((df['price']*100)/df['old_price'])).round(2) 

создаем столбец показывающий скидку в процентах

In [ ]:
df['category'] = df['soup_name'].apply(category)

Создаем столбец показывающий категорию товаров

In [ ]:
df['subcategory'] = df['soup_name'].apply(subcategory)

создаем столбец показывающий подкатегорию товаров

In [ ]:
df['group'] = df['soup_name'].apply(group)

создаем столбец показывающий группу товаров

In [ ]:
df.loc[df['subcategory']=='monitory','group'] = 'monitory'

в подкатегории мониторы меняем названия групп на мониторы

In [ ]:
df.head()

In [ ]:
df['category'].unique()

In [ ]:
df['subcategory'].unique()

In [ ]:
df.group.unique()

проссматриваем названия категорий подкатегорий и групп товаров

In [ ]:
df.loc[df['group'].isna(),'group'] = 'No_group'

удалеем из столбца group все значения None

In [ ]:
def count_group(x):
    return df[df['group']==x]['id'].count()    

In [ ]:
tqdm.pandas()
df['count_group'] = df['group'].progress_apply(count_group)

создание столбца с количеством товара в группе

In [ ]:
def count_group_presence(x):
    return df[(df['availability']=='В наличии')&(df['group']==x)]['id'].count()   

In [ ]:
tqdm.pandas()
df['count_group_presence'] = df['group'].progress_apply(count_group_presence)

создание столбца с количеством товара, который есть в наличии, в группе

In [ ]:
def dif_price(x):    
    max_price = df[(df['availability']=='В наличии')&(df['group']==x)]['price'].max()
    min_price = df[(df['availability']=='В наличии')&(df['group']==x)]['price'].min() 
    return max_price / min_price

In [ ]:
tqdm.pandas()
df['dif_price'] = df['group'].progress_apply(dif_price)

создам столбец с отношением самой дорогой цены в группе к самой дешевой.

In [ ]:
def brends(x):
    xx = x.replace('\xa0', ' ')
    xx = xx.split(' ')    
    for i in range(len(xx)-1):
        if xx[i].lower() in brend_low:
            return xx[i].lower()
        elif (str(xx[i].lower())+' '+str(xx[i+1].lower())) in brend_low:
            return (str(xx[i].lower())+' '+str(xx[i+1].lower()))    
    else:    
        return 'другой'

In [ ]:
tqdm.pandas()
df['brend'] = df['title'].progress_apply(brends)

In [ ]:
df['group_prise'] = df.groupby('group')['price'].transform(lambda x: pd.cut(x, bins = 4, labels=["бюджет", "стандарт", "элит", "топ"]))

In [ ]:
today = pd.to_datetime('today').normalize()
today = today.to_pydatetime()
df.to_csv('csv/hi_tech_sity_'+str(today.year)+'_'+str(today.month)+'_'+str(today.day)+'.csv', index=False)  

# Добавление остатков по магазинам

def count_sity(x):
    try:
        reqx = requests.get(x)
        soup = bs(reqx.text, 'html.parser')    
        element = soup.findAll('div', class_='ty-warehouses-store__availability')
        elem_2 = soup.findAll('div', class_='ty-breadcrumbs clearfix')
        name = soup.findAll('span', class_='ty-breadcrumbs__current')
        data = []
        if len(element) == 14:
            data.append(element[0].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))    
            data.append(element[1].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
            data.append(element[2].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
            data.append(element[3].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
            data.append(element[4].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0')) 
            data.append(element[5].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
            data.append(element[6].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
            data.append(element[7].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
            data.append(element[8].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
            data.append(element[9].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
            data.append(element[10].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
            data.append(element[11].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
            data.append(element[12].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
            data.append(element[13].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
            data.append(0)
            if name[0].text.find('/')<0:
                ss = elem_2[0].text
            else:
                ss = elem_2[0].text[:elem_2[0].text.rfind('/')] + ' ' + elem_2[0].text[elem_2[0].text.rfind('/')+1:]           

            if len(ss.split('/')) == 3:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[2])
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 4:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[2])  
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 5:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[3]) 
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 6:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[-3])  
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 7:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[-4])  
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 8:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[-5])  
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 9:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[-6]) 
                data.append(len(ss.split('/')))
            return data
        else:
            data.append(0)
            data.append(0)
            data.append(0)
            data.append(0)
            data.append(0)
            data.append(0)
            data.append(0)
            data.append(0)
            data.append(0)
            data.append(0)
            data.append(0)
            data.append(0)
            data.append(0)
            data.append(0)
            data.append(1)
            if len(ss.split('/')) == 3:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[2])
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 4:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[2])  
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 5:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[3]) 
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 6:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[-3])  
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 7:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[-4])  
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 8:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[-5])  
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 9:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[-6]) 
                data.append(len(ss.split('/')))
            return data
    except:
        pass


In [ ]:
def count_sity(x):
    try:
        reqx = requests.get(x)
        soup = bs(reqx.text, 'html.parser')    
        element = soup.findAll('div', class_='ty-warehouses-store__availability')
        elem_2 = soup.findAll('div', class_='ty-breadcrumbs clearfix')
        name = soup.findAll('span', class_='ty-breadcrumbs__current')
        data = []
        try:
            data.append(element[0].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[1].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[2].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[3].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[4].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[5].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[6].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[7].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[8].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[9].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[10].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[11].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[12].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[13].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:        
            if name[0].text.find('/')<0:
                ss = elem_2[0].text
            else:
                ss = elem_2[0].text[:elem_2[0].text.rfind('/')] + ' ' + elem_2[0].text[elem_2[0].text.rfind('/')+1:]           

            if len(ss.split('/')) == 3:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[2])
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 4:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[2])  
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 5:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[3]) 
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 6:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[-3])  
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 7:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[-4])  
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 8:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[-5])  
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 9:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[-6]) 
                data.append(len(ss.split('/')))
            return data
        except:
            data.append(np.NaN)
            data.append(np.NaN)
            data.append(np.NaN)
            data.append(np.NaN)
            return data
    except:
        pass

In [ ]:
tqdm.pandas()
df['count_sity']= df['soup_name'].progress_apply(count_sity)

In [ ]:
df['Тир_Юности_14А'] = df['count_sity'].str[0]
df['Ц_Тир_25_Октября_69'] = df['count_sity'].str[1]
df['Бенд_Спортивная_5'] = df['count_sity'].str[2]
df['Бенд_Советская_5'] = df['count_sity'].str[3]
df['Бенд_Советская_8'] = df['count_sity'].str[4]
df['СЛОБОДЗЕЯ'] = df['count_sity'].str[5]
df['ДНЕСТРОВСК'] = df['count_sity'].str[6]
df['Первомайск'] = df['count_sity'].str[7]
df['Григ_КМаркса_181А'] = df['count_sity'].str[8]
df['Григ_КМаркса_172'] = df['count_sity'].str[9]
df['ДУБОССАРЫ'] = df['count_sity'].str[10]
df['Рыбн_Вальченко_43'] = df['count_sity'].str[11]
df['Рыбн_Кирова_135'] = df['count_sity'].str[12]
df['Каменка'] = df['count_sity'].str[13]
df['_category'] = df['count_sity'].str[15]
df['_subcategory'] = df['count_sity'].str[16]
df['_group'] = df['count_sity'].str[17]
df['len'] = df['count_sity'].str[18]

In [ ]:
df = df[~(df['Тир_Юности_14А'].isnull())]

In [ ]:
df['Тир_Юности_14А'] = df['Тир_Юности_14А'].astype('int')
df['Ц_Тир_25_Октября_69'] = df['Ц_Тир_25_Октября_69'].astype('int')
df['ТИРАСПОЛЬ'] = df['Тир_Юности_14А'] + df['Ц_Тир_25_Октября_69']
df['Бенд_Спортивная_5'] = df['Бенд_Спортивная_5'].astype('int')
df['Бенд_Советская_5'] = df['Бенд_Советская_5'].astype('int')
df['Бенд_Советская_8'] = df['Бенд_Советская_8'].astype('int')
df['БЕНДЕРЫ'] = df['Бенд_Спортивная_5'] + df['Бенд_Советская_5'] + df['Бенд_Советская_8']
df['СЛОБОДЗЕЯ'] = df['СЛОБОДЗЕЯ'].astype('int')
df['ДНЕСТРОВСК'] = df['ДНЕСТРОВСК'].astype('int')
df['Первомайск'] = df['Первомайск'].astype('int')
df['Григ_КМаркса_181А'] = df['Григ_КМаркса_181А'].astype('int')
df['Григ_КМаркса_172'] = df['Григ_КМаркса_172'].astype('int')
df['Григориополь'] = df['Григ_КМаркса_181А'] + df['Григ_КМаркса_172']
df['ДУБОССАРЫ'] = df['ДУБОССАРЫ'].astype('int')
df['Рыбн_Вальченко_43'] = df['Рыбн_Вальченко_43'].astype('int')
df['Рыбн_Кирова_135'] = df['Рыбн_Кирова_135'].astype('int')
df['РЫБНИЦА'] = df['Рыбн_Вальченко_43'] + df['Рыбн_Кирова_135']
df['Каменка'] = df['Каменка'].astype('int')

In [ ]:
df.drop(['count_sity'], axis= 1 , inplace= True )

In [ ]:
preprocessing(df)

In [ ]:
separator_categori = ['Товар без акции', 'Акция', 'Товар по акции']

In [ ]:
for i in separator_categori:
    df.loc[df['_categori'] == i,'_subcategory'] = i
    df.loc[df['_categori'] == i,'_group'] = i

In [ ]:
today = pd.to_datetime('today').normalize()
today = today.to_pydatetime()
df.to_csv('csv/hi_tech_end_'+str(today.year)+'_'+str(today.month)+'_'+str(today.day)+'.csv', index=False)  

In [ ]:
df.head(2)

In [ ]:
df._group.unique()

In [ ]:
df._subcategory.unique()

In [ ]:
dd = ['Батарейки и зарядные устройства', 'Крепления для тв',
       'Аудиосистемы', 'Смартфоны и аксессуары', 'Мультимедиа',
       'Приставки для ТВ', 'Гаджеты', 'Телефоны dect и проводные',
       'Apple', 'Видеотехника', 'Товар без акции', 'Умный дом с Яндекс',
       'Телевизоры', 'Пылесосы и аксессуары', 'Техника karcher',
       'Аксессуары для мелкой бытовой техники',
       'Товары для ухода за одеждой', 'Минимойки',
       'Приготовление напитков', 'Мелкая техника для кухни',
       'Техника для красоты', 'Стиральные и сушильные машины',
       'Крупная техника для кухни', 'Техника для здоровья',
       'Климатическая техника', 'Установка бытовой техники',
       'Приготовление пищи', 'Техника для дома', 'Накопители',
       'Сетевое оборудование', 'Аксессуары', 'Расходные материалы',
       'Периферийные устройства', 'Оргтехника', 'Комплектующие',
       'Крепления', 'Электропитание', 'Игровые приставки',
       'Товар по акции', 'Мониторы', 'Проекторы и экраны',
       'Готовые решения', 'Мебель для офиса', 'Электроинструменты',
       'Сервировка стола', 'Кухонные принадлежности', 'Освещение',
       'Посуда для духовок и микроволновых печей', 'Бытовая химия']

In [ ]:
for i in dd:
    t = df[df['_subcategory']==i]
    print(i)
    print(t._group.unique())
    print('-'*100)

In [ ]:
t._group.unique()

In [ ]:
df.head(2)

In [ ]:
df.len.unique()

In [ ]:
df[df['len'] == 9]